- Github:https://github.com/Sammat2507/proyecto-sprint-8

- El objetivo de este proyecto es entrenar un modelo para que decida que plan de celular (ultra o smart) es mejor para cada persona con el menor margen de error posible. Todo esto a partir de un dataset previamente procesado y analizado. 

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
users_behavior = pd.read_csv('users_behavior.csv')

In [3]:
users_behavior.info()
print(users_behavior.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
5   58.0   344.56      21.0  15823.37         0
6   57.0   431.64      20.0   3738.90         1
7   15.0   132.40       6.0  21911.60         0
8    7.0    43.39       3.0   2538.67         1
9   90.0   665.41      38.0  17358.61         0


- En este caso creo que tal vez podemos convertir tanto calls como messages a int pero como en la descripcion del proyecto se dice que nos saltemos la parte del procesamiento de datos no se si hacerlo.

In [4]:
df_train_valid, df_test = train_test_split(users_behavior, test_size=0.20, random_state=12345)

df_train, df_valid = train_test_split(df_train_valid, test_size=0.25, random_state=12345)

In [5]:
train_features = df_train.drop(['is_ultra'], axis=1)
train_target = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

- Dividimos el df original en varios porcentajes de informacion y les damos un nombre dependiendo de su objetivo.

In [6]:
best_score = 0
best_est = 0
for est in range(1,11):
    model_random_forest = RandomForestClassifier(random_state=54321, n_estimators=est) 
    model_random_forest.fit(train_features, train_target)
    score = model_random_forest.score(features_valid, target_valid) 
    if score > best_score:
        best_score = score
        best_est = est

print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

La exactitud del mejor modelo en el conjunto de validación (n_estimators = 10): 0.7853810264385692


In [7]:
model_logic = LogisticRegression(solver='liblinear', random_state=54321)
model_logic.fit(train_features, train_target)
score_train = model_logic.score(train_features, train_target)
print(score_train)

0.700207468879668


In [8]:
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=54321, max_depth=depth)
    model_tree.fit(train_features, train_target)
    train_predictions = model_tree.predict(train_features)
    test_predictions = model_tree.predict(features_test)
    print("Exactitud de max_depth igual a", depth)
    print("Conjunto de entrenamiento:", accuracy_score(train_target, train_predictions))
    print("Conjunto de prueba:", accuracy_score(target_test, test_predictions))
    print()

Exactitud de max_depth igual a 1
Conjunto de entrenamiento: 0.758298755186722
Conjunto de prueba: 0.7480559875583204

Exactitud de max_depth igual a 2
Conjunto de entrenamiento: 0.79201244813278
Conjunto de prueba: 0.7838258164852255

Exactitud de max_depth igual a 3
Conjunto de entrenamiento: 0.8117219917012448
Conjunto de prueba: 0.7869362363919129

Exactitud de max_depth igual a 4
Conjunto de entrenamiento: 0.8205394190871369
Conjunto de prueba: 0.7869362363919129

Exactitud de max_depth igual a 5
Conjunto de entrenamiento: 0.8278008298755186
Conjunto de prueba: 0.7900466562986003

Exactitud de max_depth igual a 6
Conjunto de entrenamiento: 0.8335062240663901
Conjunto de prueba: 0.7838258164852255

Exactitud de max_depth igual a 7
Conjunto de entrenamiento: 0.8506224066390041
Conjunto de prueba: 0.7900466562986003

Exactitud de max_depth igual a 8
Conjunto de entrenamiento: 0.8661825726141079
Conjunto de prueba: 0.7791601866251944

Exactitud de max_depth igual a 9
Conjunto de entren

- Entrenamos y hacemos pruebas con los tres modelos de problemas de clasificacion conocidos para despues ver cual tiene menos errores trabajando con este problema. 

In [9]:
score_test = model_logic.score(features_test, target_test)
print(score_test)

0.6967340590979783


In [10]:
dummy = DummyClassifier(strategy='constant', constant=0)
dummy.fit(train_features, train_target)
dummy_score = dummy.score(features_test, target_test)
print(dummy_score)

0.6951788491446346


- Comparo la calidad del modelo que seleccionamos con un modelo simple que escoje siempre la misma clase.